In [2]:
import requests
from requests.auth import HTTPBasicAuth

def generarToken(usuario):
    url = "https://apisolicitudesonlinepre.cinconet.local/api/Auth/login"
    username = usuario
    password = "tu_contraseña"
    response = requests.post(url, auth=HTTPBasicAuth(username, password), verify=False)
    return response.json().get("validToken")

In [3]:
import requests

mail="vchumillas@megamedia.es"
print(mail)
usuario, dominio = mail.split('@')

print("Usuario:", usuario)
print("Dominio:", dominio)

validToken = generarToken(mail)
url = f"https://apisolicitudesonline.cinconet.local/api/Gestion/Usuario/Directorio/{usuario}"
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'Authorization': f'Bearer {validToken}'
}

response = requests.get(url, headers=headers, verify=False)
print(response.text)


vchumillas@megamedia.es
Usuario: vchumillas
Dominio: megamedia.es


c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonlinepre.cinconet.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{"mail":"vchumillas@megamedia.es","mail_2":"vchumillas@megamedia.es","clasificacionProfesional":"TEC","rolId":0,"rol":null,"autorizador":null,"activo":false,"correo":null,"empresa":"MEGAMEDIA TELEVISION, S.L.","empresaId":31,"departamento":"DESARROLLO, PRODUCTO E INNOVACION","localizacion":"Edificio B Planta 4","numeroEmpleado":310536,"telefono":null,"esAutorizador":false,"delegadosId":null,"delegados":null,"isApprover":false,"canDelegate":false,"esAuditor":false,"esAdministrado":false,"id":0,"login":"vchumillas","nombre":"VICTOR","apellidos":"CHUMILLAS HERNANDO"}


c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonline.cinconet.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [24]:
def obtener_ids(programa_nombre: str, rol_nombre: str):
    niveles_con_ids = {
        598: {
            "descripcion": "Supervivientes",
            "roles": {
                433: "Redactor",
                434: "Realizador/Ayudante de Realización",
                435: "Editor de vídeo",
                436: "Dirección",
                437: "Postproducción",
                438: "Tituladora"
            }
        },
        599: {
            "descripcion": "Fiesta",
            "roles": {
                439: "Redactor",
                458: "Realizador/Ayudante de Realización",
                459: "Editor de vídeo",
                460: "Dirección",
                461: "Postproducción",
                462: "Tituladora"
            }
        },
        600: {
            "descripcion": "La vida sin filtros",
            "roles": {
                463: "Redactor",
                464: "Realizador/Ayudante de Realización",
                465: "Editor de vídeo",
                466: "Dirección",
                467: "Postproducción",
                468: "Tituladora"
            }
        },
        601: {
            "descripcion": "Así es la vida",
            "roles": {
                456: "Tituladora",
                457: "Postproducción",
                469: "Redactor",
                470: "Realizador/Ayudante de Realización",
                471: "Editor de vídeo",
                472: "Dirección"
            }
        },
        602: {
            "descripcion": "Vamos a ver",
            "roles": {
                440: "Editor de vídeo",
                441: "Dirección",
                442: "Postproducción",
                443: "Tituladora",
                446: "Realizador/Ayudante de Realización",
                455: "Redactor"
            }
        },
        603: {
            "descripcion": "La mirada crítica",
            "roles": {
                444: "Redactor",
                445: "Realizador/Ayudante de Realización",
                447: "Editor de vídeo",
                448: "Postproducción",
                449: "Dirección",
                454: "Dirección"
            }
        },
        604: {
            "descripcion": "TardeAR",
            "roles": {
                450: "Redactor",
                451: "Realizador/Ayudante de Realización",
                452: "Editor de vídeo",
                453: "Dirección",
                473: "Postproducción",
                474: "Dirección"
            }
        },
        644: {
            "descripcion": "El diario de Jorge",
            "roles": {
                522: "Redactor",
                523: "Realizador/Ayudante de Realización",
                524: "Editor de vídeo",
                525: "Dirección",
                526: "Postproducción",
                527: "Media Manager"
            }
        },
        682: {
            "descripcion": "El Programa de Ana Rosa",
            "roles": {
                551: "Redactor",
                552: "Realizador/Ayudante de Realización",
                553: "Editor de vídeo",
                554: "Dirección",
                555: "Postproducción",
                556: "Media Manager"
            }
        }
    }

    for nivel1_id, nivel_data in niveles_con_ids.items():
        if nivel_data["descripcion"].lower() == programa_nombre.lower():
            for nivel2_id, nombre_rol in nivel_data["roles"].items():
                if nombre_rol.lower() == rol_nombre.lower():
                    id_programa=nivel1_id
                    id_rol=nivel2_id

    return id_programa, id_rol  # Si no se encuentra

# Ejemplo de uso
#programa = "Así es la vida"
#rol = "Redactor"

# nivel1_id, nivel2_id = obtener_ids(programa, rol)
# print(f"Nivel1 ID: {nivel1_id}, Nivel2 ID: {nivel2_id}")


In [28]:
import streamlit as st
import asyncio
import requests
from requests.auth import HTTPBasicAuth
from datetime import datetime
import openai
import time
import json
import re
import urllib

def getDataUserByMail(mail, is_destinatario=False):
    """
    Obtiene los datos del usuario a partir de su mail llamando a la API.
    Si is_destinatario es True, no incluye autorizadorId ni autorizadorApoyoId.
    """
    print(mail)
    usuario, dominio = mail.split('@')

    print("Usuario:", usuario)
    print("Dominio:", dominio)

    validToken = generarToken(mail)
    url = f"https://apisolicitudesonline.cinconet.local/api/Gestion/Usuario/Directorio/{usuario}"
    headers = {
        'Content-Type': 'application/json;charset=UTF-8',
        'Authorization': f'Bearer {validToken}'
    }

    response = requests.get(url, headers=headers, verify=False)
    if response.status_code != 200:
        return None  # Retorna None si hay un error

    data = response.json()
    nombre = data.get("nombre", "")
    apellidos = data.get("apellidos", "")
    empresa = data.get("empresa", "")
    # Extraer valores
    correo = mail
    login = usuario

    validToken = generarToken(correo)
    url = f"https://apisolicitudesonlinepre.cinconet.local/api/Solicitud/Ultima"
    headers['Authorization'] = f'Bearer {validToken}'

    response = requests.get(url, headers=headers, verify=False)
    if response.status_code != 200:
        return None  # Retorna None si hay un error

    data = response.json()

    # Construir el diccionario base
    user_data = {
        "nombre": nombre,
        "apellidos": apellidos,
        "login": login,
        "email": correo,
        "telefono": data.get("telefono"),
        "departamento": data.get("departamento"),
        "empresa": empresa,
        "centro": data.get("centro"),
        "edificio": data.get("edificio"),
        "planta": data.get("planta")
    }
    
    # Si no es destinatario, agregar autorizadorId y autorizadorApoyoId
    if not is_destinatario:
        user_data["autorizadorId"] = data.get("autorizadorId")
        user_data["autorizadorApoyoId"] = data.get("autorizadorApoyoId")
    
    return user_data

def generar_jsonMails_externos(principal_mail, destinatarios_externos):
    """
    Genera el JSON con los datos del usuario principal y los destinatarios.
    """
    # Obtener datos del usuario principal
    principal_data = getDataUserByMail(principal_mail)
    if not principal_data:
        return {"error": f"No se encontró el usuario principal externos{principal_mail}"}

    formatted_externos = []
    for ext in destinatarios_externos:
            ext_payload = {
                "nombre": ext["nombre"],
                "apellidos": ext["apellidos"],
                "login": "",
                "departamento": ext["departamento"],
                "empresa": ext["empresa"],
                "centro": ext["centro"],
                "edificio": "" if ext.get("edificio", "").lower() == "ninguno" else ext["edificio"],
                "planta": "" if ext.get("planta", "").lower() == "ninguno" else ext["planta"]
            }
            formatted_externos.append(ext_payload)

    # Construir el JSON final
    resultado = {
        "autorizadorId": principal_data["autorizadorId"],
        "autorizadorApoyoId": principal_data["autorizadorApoyoId"],
        "telefono": principal_data["telefono"],
        "departamento": principal_data["departamento"],
        "destinatarios": formatted_externos,
        "observaciones": "test",
        "justificacion": "Test",
        "productoraId": None,
        "programa": "",
        "centro": principal_data["centro"],
        "edificio": principal_data["edificio"],
        "planta": principal_data["planta"]
    }

    return resultado

def generar_jsonMails(principal_mail, destinatarios_mails):
    """
    Genera el JSON con los datos del usuario principal y los destinatarios.
    """
    # Obtener datos del usuario principal
    principal_data = getDataUserByMail(principal_mail)
    if not principal_data:
        return {"error": f"No se encontró el usuario principal{principal_mail}"}

    # Obtener datos de los destinatarios (si la lista no está vacía)
    destinatarios = [getDataUserByMail(dest_mail, is_destinatario=True) for dest_mail in destinatarios_mails]
    destinatarios = [d for d in destinatarios if d]  # Filtrar valores None

    # Construir el JSON final
    resultado = {
        "autorizadorId": principal_data["autorizadorId"],
        "autorizadorApoyoId": principal_data["autorizadorApoyoId"],
        "telefono": principal_data["telefono"],
        "departamento": principal_data["departamento"],
        "destinatarios": destinatarios,
        "observaciones": "test",
        "justificacion": "Test",
        "productoraId": None,
        "programa": "",
        "centro": principal_data["centro"],
        "edificio": principal_data["edificio"],
        "planta": principal_data["planta"]
    }

    return resultado

def formatear_fecha(fecha_str):
    try:
        fecha_obj = datetime.fromisoformat(fecha_str[:-1])
        return fecha_obj.strftime("%Y-%m-%d %Hh")
    except:
        return "Fecha inválida"

def generarToken(usuario):
    url = "https://apisolicitudesonlinepre.cinconet.local/api/Auth/login"
    username = usuario
    password = "tu_contraseña"
    response = requests.post(url, auth=HTTPBasicAuth(username, password), verify=False)
    return response.json().get("validToken")

def create_solicitud_Inews(tipoAcceso,permisos,observaciones="",destinatarios_internos="", destinatarios_externos=None):
    niveles_con_ids = {
        598: {
            "descripcion": "Supervivientes",
            "roles": {
                433: "Redactor",
                434: "Realizador/Ayudante de Realización",
                435: "Editor de vídeo",
                436: "Dirección",
                437: "Postproducción",
                438: "Tituladora"
            }
        },
        599: {
            "descripcion": "Fiesta",
            "roles": {
                439: "Redactor",
                458: "Realizador/Ayudante de Realización",
                459: "Editor de vídeo",
                460: "Dirección",
                461: "Postproducción",
                462: "Tituladora"
            }
        },
        600: {
            "descripcion": "La vida sin filtros",
            "roles": {
                463: "Redactor",
                464: "Realizador/Ayudante de Realización",
                465: "Editor de vídeo",
                466: "Dirección",
                467: "Postproducción",
                468: "Tituladora"
            }
        },
        601: {
            "descripcion": "Así es la vida",
            "roles": {
                456: "Tituladora",
                457: "Postproducción",
                469: "Redactor",
                470: "Realizador/Ayudante de Realización",
                471: "Editor de vídeo",
                472: "Dirección"
            }
        },
        602: {
            "descripcion": "Vamos a ver",
            "roles": {
                440: "Editor de vídeo",
                441: "Dirección",
                442: "Postproducción",
                443: "Tituladora",
                446: "Realizador/Ayudante de Realización",
                455: "Redactor"
            }
        },
        603: {
            "descripcion": "La mirada crítica",
            "roles": {
                444: "Redactor",
                445: "Realizador/Ayudante de Realización",
                447: "Editor de vídeo",
                448: "Postproducción",
                449: "Dirección",
                454: "Dirección"
            }
        },
        604: {
            "descripcion": "TardeAR",
            "roles": {
                450: "Redactor",
                451: "Realizador/Ayudante de Realización",
                452: "Editor de vídeo",
                453: "Dirección",
                473: "Postproducción",
                474: "Dirección"
            }
        },
        644: {
            "descripcion": "El diario de Jorge",
            "roles": {
                522: "Redactor",
                523: "Realizador/Ayudante de Realización",
                524: "Editor de vídeo",
                525: "Dirección",
                526: "Postproducción",
                527: "Media Manager"
            }
        },
        682: {
            "descripcion": "El Programa de Ana Rosa",
            "roles": {
                551: "Redactor",
                552: "Realizador/Ayudante de Realización",
                553: "Editor de vídeo",
                554: "Dirección",
                555: "Postproducción",
                556: "Media Manager"
            }
        }
    }

    for nivel1_id, nivel_data in niveles_con_ids.items():
        if nivel_data["descripcion"].lower() == tipoAcceso.lower():
            for nivel2_id, nombre_rol in nivel_data["roles"].items():
                if nombre_rol.lower() == permisos.lower():
                    id_programa=nivel1_id
                    id_rol=nivel2_id

    lista_destinatarios=[]
    nombres=""
    if destinatarios_internos:
        destinatarios=destinatarios_internos
        destinatarios = [
            "vchumillas@megamedia.es" if d.strip().lower() == "user" else d.strip()
            for d in destinatarios
        ]
        print(f"input usuarios:{destinatarios_internos}")
        user = "vchumillas@megamedia.es"
        validToken = generarToken(user)
        
        url = "https://apisolicitudesonlinepre.cinconet.local/api/Recurso/Carrito"


        payload_dict = {
        "idRecurso": 0,
        "idRecursoCatalogo": 340,
        "nombre": "INEWS (Usuarios red de programas (RDP) - Telecinco)",
        "descripcion": "INEWS (Usuarios red de programas Telecinco)",
        "urlImagen": "NO_IMAGE",
        "coste": 0.00,
        "activo": True,
        "nivelesRecurso": [
            {
                "id": 28225,
                "nivel1Id": id_programa,
                "nivel1": None,
                "nivel2Id": id_rol,
                "nivel2": None,
                "nivel3": None
            }
        ],
        "niveles": [
            {
                "id": 598,
                "descripcion": "Supervivientes",
                "nivel2": [
                    {
                        "id": 433,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 434,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 435,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 436,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 437,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 438,
                        "descripcion": "Tituladora",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 599,
                "descripcion": "Fiesta",
                "nivel2": [
                    {
                        "id": 439,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 458,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 459,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 460,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 461,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 462,
                        "descripcion": "Tituladora",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 600,
                "descripcion": "La vida sin filtros",
                "nivel2": [
                    {
                        "id": 463,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 464,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 465,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 466,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 467,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 468,
                        "descripcion": "Tituladora",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 601,
                "descripcion": "Así es la vida",
                "nivel2": [
                    {
                        "id": 456,
                        "descripcion": "Tituladora",
                        "nivel3": ""
                    },
                    {
                        "id": 457,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 469,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 470,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 471,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 472,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 602,
                "descripcion": "Vamos a ver",
                "nivel2": [
                    {
                        "id": 440,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 441,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 442,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 443,
                        "descripcion": "Tituladora",
                        "nivel3": ""
                    },
                    {
                        "id": 446,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 455,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 603,
                "descripcion": "La mirada crítica",
                "nivel2": [
                    {
                        "id": 444,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 445,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 447,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 448,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 449,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 454,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 604,
                "descripcion": "TardeAR",
                "nivel2": [
                    {
                        "id": 450,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 451,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 452,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 453,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 473,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 474,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 644,
                "descripcion": "El diario de Jorge",
                "nivel2": [
                    {
                        "id": 522,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 523,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 524,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 525,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 526,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 527,
                        "descripcion": "Media Manager",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 682,
                "descripcion": "El Programa de Ana Rosa",
                "nivel2": [
                    {
                        "id": 551,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 552,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 553,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 554,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 555,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 556,
                        "descripcion": "Media Manager",
                        "nivel3": ""
                    }
                ]
            }
        ],
        "campos": [
            {
                "id": 113601,
                "maestroId": 10,
                "valor": observaciones or None,
                "recursoId": 0,
                "obligatorio": False
            },
            {
                "id": 113602,
                "maestroId": 12,
                "valor": None,
                "recursoId": 0,
                "obligatorio": False
            }
        ],
        "baja": False,
        "bajaCatalogo": True,
        "descargaPlano": False,
        "nivel1": "Tipo de acceso",
        "obligatorioNivel1": True,
        "nivel2": "Permisos",
        "obligatorioNivel2": True,
        "nivel3": None,
        "obligatorioNivel3": False
    }

        headers = {
            'Content-Type': 'application/json;charset=UTF-8',
            'Authorization': f'Bearer {validToken}'
        }

        try:
            response = requests.post(url, headers=headers, json=payload_dict, verify=False)
            response.raise_for_status()

            result = response.json()
            solicitud_id = result.get("id", "N/A")
            lista_destinatarios = [
                correo.strip() for correo in destinatarios
                if correo.strip() and "@" in correo.strip()
            ]
            print(f"input cribado:{str(lista_destinatarios)}")
            #lista_targets = [item.strip() for item in destinatarios.split(",")]
            data = generar_jsonMails(user, lista_destinatarios)
            print(f"data:{str(data)}")
            url = "https://apisolicitudesonlinepre.cinconet.local/api/Solicitud"
            headers = {
                    'Content-Type': 'application/json;charset=UTF-8',
                    'Authorization': f'Bearer {validToken}'
                }
            response = requests.post(url, data=json.dumps(data), headers=headers, verify=False)
        except requests.exceptions.RequestException as e:
            resultado+= f"⚠️ Error al enviar la solicitud de office365: {str(e)}"
    if destinatarios_externos:
        destinatarios=destinatarios_externos
        print(f"input usuarios:{destinatarios_externos}")
        user = "vchumillas@megamedia.es"
        validToken = generarToken(user)
        
        url = "https://apisolicitudesonlinepre.cinconet.local/api/Recurso/Carrito"

        payload_dict = {
        "idRecurso": 0,
        "idRecursoCatalogo": 340,
        "nombre": "INEWS (Usuarios red de programas (RDP) - Telecinco)",
        "descripcion": "INEWS (Usuarios red de programas Telecinco)",
        "urlImagen": "NO_IMAGE",
        "coste": 0.00,
        "activo": True,
        "nivelesRecurso": [
            {
                "id": 28225,
                "nivel1Id": id_programa,
                "nivel1": None,
                "nivel2Id": id_rol,
                "nivel2": None,
                "nivel3": None
            }
        ],
        "niveles": [
            {
                "id": 598,
                "descripcion": "Supervivientes",
                "nivel2": [
                    {
                        "id": 433,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 434,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 435,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 436,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 437,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 438,
                        "descripcion": "Tituladora",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 599,
                "descripcion": "Fiesta",
                "nivel2": [
                    {
                        "id": 439,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 458,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 459,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 460,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 461,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 462,
                        "descripcion": "Tituladora",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 600,
                "descripcion": "La vida sin filtros",
                "nivel2": [
                    {
                        "id": 463,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 464,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 465,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 466,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 467,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 468,
                        "descripcion": "Tituladora",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 601,
                "descripcion": "Así es la vida",
                "nivel2": [
                    {
                        "id": 456,
                        "descripcion": "Tituladora",
                        "nivel3": ""
                    },
                    {
                        "id": 457,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 469,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 470,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 471,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 472,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 602,
                "descripcion": "Vamos a ver",
                "nivel2": [
                    {
                        "id": 440,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 441,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 442,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 443,
                        "descripcion": "Tituladora",
                        "nivel3": ""
                    },
                    {
                        "id": 446,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 455,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 603,
                "descripcion": "La mirada crítica",
                "nivel2": [
                    {
                        "id": 444,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 445,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 447,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 448,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 449,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 454,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 604,
                "descripcion": "TardeAR",
                "nivel2": [
                    {
                        "id": 450,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 451,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 452,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 453,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 473,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 474,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 644,
                "descripcion": "El diario de Jorge",
                "nivel2": [
                    {
                        "id": 522,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 523,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 524,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 525,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 526,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 527,
                        "descripcion": "Media Manager",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 682,
                "descripcion": "El Programa de Ana Rosa",
                "nivel2": [
                    {
                        "id": 551,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 552,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 553,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 554,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 555,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 556,
                        "descripcion": "Media Manager",
                        "nivel3": ""
                    }
                ]
            }
        ],
        "campos": [
            {
                "id": 113601,
                "maestroId": 10,
                "valor": observaciones or None,
                "recursoId": 0,
                "obligatorio": False
            },
            {
                "id": 113602,
                "maestroId": 12,
                "valor": None,
                "recursoId": 0,
                "obligatorio": False
            }
        ],
        "baja": False,
        "bajaCatalogo": True,
        "descargaPlano": False,
        "nivel1": "Tipo de acceso",
        "obligatorioNivel1": True,
        "nivel2": "Permisos",
        "obligatorioNivel2": True,
        "nivel3": None,
        "obligatorioNivel3": False
    }

        headers = {
            'Content-Type': 'application/json;charset=UTF-8',
            'Authorization': f'Bearer {validToken}'
        }

        try:
            response = requests.post(url, headers=headers, json=payload_dict, verify=False)
            response.raise_for_status()

            result = response.json()
            solicitud_id = result.get("id", "N/A")
            
            #lista_targets = [item.strip() for item in destinatarios.split(",")]
            data = generar_jsonMails_externos(user, destinatarios)
            print(f"data:{str(data)}")
            url = "https://apisolicitudesonlinepre.cinconet.local/api/Solicitud"
            headers = {
                    'Content-Type': 'application/json;charset=UTF-8',
                    'Authorization': f'Bearer {validToken}'
                }
            response = requests.post(url, data=json.dumps(data), headers=headers, verify=False)
            nombres=""
            for destinatario in destinatarios:
                nombres+=destinatario["nombre"]+" "+destinatario["apellidos"] + ", "
        except requests.exceptions.RequestException as e:
            resultado+= f"⚠️ Error al enviar la solicitud de Office365: {str(e)}"

        lista_destinatarios=", ".join(lista_destinatarios)
    return (
        f"📡 Solicitud de Inews enviada correctamente:\n"
        f"- tipo de Acceso: {tipoAcceso}\n"
        f"- permisos: {permisos}\n"
        f"- observaciones: {observaciones or 'sin observaciones'}\n"
        f"- destinatarios: {nombres + str(lista_destinatarios) or 'tu'}\n"
    )

def create_solicitud_Interplay(tipoAcceso,permisos,observaciones="",destinatarios_internos="", destinatarios_externos=None):
    niveles_con_ids = {
    "Supervivientes": {
        "id": 249,
        "roles": {
            "Media manager": 160,
            "Redactor": 161,
            "Dirección": 393,
            "Editor de vídeo": 394,
            "Realizador/Ayudante de Realización": 395,
            "Postproducción": 409
        }
    },
    "Fiesta": {
        "id": 502,
        "roles": {
            "Media Manager": 251,
            "Dirección": 385,
            "Editor de vídeo": 389,
            "Redactor": 391,
            "Realizador/Ayudante de Realización": 397,
            "Postproducción": 408
        }
    },
    "La vida sin filtros": {
        "id": 534,
        "roles": {
            "Editor de vídeo": 279,
            "Redactor": 383,
            "Realizador/Ayudante de Realización": 384,
            "Dirección": 388,
            "Postproducción": 407
        }
    },
    "Así es la vida": {
        "id": 535,
        "roles": {
            "Redactor": 386,
            "Realizador/Ayudante de Realización": 387,
            "Dirección": 390,
            "Editor de vídeo": 396,
            "Postproducción": 406
        }
    },
    "Vamos A Ver": {
        "id": 568,
        "roles": {
            "Redactor": 374,
            "Guionista": 375,
            "Dirección": 376,
            "Producción": 377,
            "Postproducción": 398,
            "Dirección (2)": 403,
            "Realizador/Ayudante de Realización": 405,
            "Editor de vídeo": 411
        }
    },
    "La Mirada Crítica": {
        "id": 569,
        "roles": {
            "Redactor": 370,
            "Guionista": 371,
            "Dirección": 372,
            "Producción": 373,
            "Postproducción": 399,
            "Editor de vídeo": 400,
            "Realizador/Ayudante de Realización": 401
        }
    },
    "TardeAR": {
        "id": 570,
        "roles": {
            "Redactor": 366,
            "Guionista": 367,
            "Dirección": 368,
            "Producción": 369,
            "Realizador/Ayudante de Realización": 402,
            "Postproducción": 404,
            "Editor de vídeo": 410
        }
    },
    "Vecinos": {
        "id": 615,
        "roles": {
            "Realizador/Ayudante de Realización": 510,
            "Redactor": 511,
            "Guionista": 512,
            "Dirección": 513,
            "Producción": 514,
            "Postproducción": 515,
            "Editor de vídeo": 516
        }
    },
    "El Programa de Ana Rosa": {
        "id": 683,
        "roles": {
            "Realizador/Ayudante de Realización": 557,
            "Redactor": 558,
            "Guionista": 559,
            "Dirección": 560,
            "Producción": 561,
            "Postproducción": 562,
            "Editor de vídeo": 563
        }
    }
}

    if tipoAcceso not in niveles_con_ids:
        return f"Programa no encontrado: {tipoAcceso}"
    nivel1_id = niveles_con_ids[tipoAcceso]["id"]
    nivel2_id = niveles_con_ids[tipoAcceso]["roles"].get(permisos)
    if nivel2_id is None:
        return f"Rol no encontrado: {permisos} en el programa {tipoAcceso}"
    id_programa=nivel1_id
    id_rol=nivel2_id

    lista_destinatarios=[]
    nombres=""
    if destinatarios_internos:
        destinatarios=destinatarios_internos
        destinatarios = [
            "vchumillas@megamedia.es" if d.strip().lower() == "user" else d.strip()
            for d in destinatarios
        ]
        print(f"input usuarios:{destinatarios_internos}")
        user = "vchumillas@megamedia.es"
        validToken = generarToken(user)
        
        url = "https://apisolicitudesonlinepre.cinconet.local/api/Recurso/Carrito"


        payload_dict = {
        "idRecurso": 0,
        "idRecursoCatalogo": 161,
        "nombre": "Interplay (Usuarios red de programas (RDP) - Telecinco)",
        "descripcion": "Inteplay (Usuarios red de programas Telecinco)",
        "urlImagen": "NO_IMAGE",
        "coste": 0.00,
        "activo": True,
        "nivelesRecurso": [
            {
                "id": 28227,
                "nivel1Id": id_programa,
                "nivel1": None,
                "nivel2Id": id_rol,
                "nivel2": None,
                "nivel3": None
            }
        ],
        "niveles": [
            {
                "id": 249,
                "descripcion": "Supervivientes",
                "nivel2": [
                    {
                        "id": 160,
                        "descripcion": "Media manager",
                        "nivel3": ""
                    },
                    {
                        "id": 161,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 393,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 394,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 395,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 409,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 502,
                "descripcion": "Fiesta",
                "nivel2": [
                    {
                        "id": 251,
                        "descripcion": "Media Manager",
                        "nivel3": ""
                    },
                    {
                        "id": 385,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 389,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 391,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 397,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 408,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 534,
                "descripcion": "La vida sin filtros",
                "nivel2": [
                    {
                        "id": 279,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 383,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 384,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 388,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 407,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 535,
                "descripcion": "Así es la vida",
                "nivel2": [
                    {
                        "id": 386,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 387,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 390,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 396,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 406,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 568,
                "descripcion": "Vamos A Ver",
                "nivel2": [
                    {
                        "id": 374,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 375,
                        "descripcion": "Guionista",
                        "nivel3": ""
                    },
                    {
                        "id": 376,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 377,
                        "descripcion": "Producción",
                        "nivel3": ""
                    },
                    {
                        "id": 398,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 403,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 405,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 411,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 569,
                "descripcion": "La Mirada Crítica",
                "nivel2": [
                    {
                        "id": 370,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 371,
                        "descripcion": "Guionista",
                        "nivel3": ""
                    },
                    {
                        "id": 372,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 373,
                        "descripcion": "Producción",
                        "nivel3": ""
                    },
                    {
                        "id": 399,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 400,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 401,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 570,
                "descripcion": "TardeAR",
                "nivel2": [
                    {
                        "id": 366,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 367,
                        "descripcion": "Guionista",
                        "nivel3": ""
                    },
                    {
                        "id": 368,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 369,
                        "descripcion": "Producción",
                        "nivel3": ""
                    },
                    {
                        "id": 402,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 404,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 410,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 615,
                "descripcion": "Vecinos",
                "nivel2": [
                    {
                        "id": 510,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 511,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 512,
                        "descripcion": "Guionista",
                        "nivel3": ""
                    },
                    {
                        "id": 513,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 514,
                        "descripcion": "Producción",
                        "nivel3": ""
                    },
                    {
                        "id": 515,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 516,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 683,
                "descripcion": "El Programa de Ana Rosa",
                "nivel2": [
                    {
                        "id": 557,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 558,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 559,
                        "descripcion": "Guionista",
                        "nivel3": ""
                    },
                    {
                        "id": 560,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 561,
                        "descripcion": "Producción",
                        "nivel3": ""
                    },
                    {
                        "id": 562,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 563,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    }
                ]
            }
        ],
        "campos": [
            {
                "id": 113611,
                "maestroId": 10,
                "valor": observaciones or None,
                "recursoId": 0,
                "obligatorio": False
            },
            {
                "id": 113612,
                "maestroId": 12,
                "valor": None,
                "recursoId": 0,
                "obligatorio": False
            }
        ],
        "baja": False,
        "bajaCatalogo": True,
        "descargaPlano": False,
        "nivel1": "Tipo de acceso",
        "obligatorioNivel1": True,
        "nivel2": "Permisos",
        "obligatorioNivel2": True,
        "nivel3": None,
        "obligatorioNivel3": False
    }

        headers = {
            'Content-Type': 'application/json;charset=UTF-8',
            'Authorization': f'Bearer {validToken}'
        }

        try:
            response = requests.post(url, headers=headers, json=payload_dict, verify=False)
            response.raise_for_status()

            result = response.json()
            solicitud_id = result.get("id", "N/A")
            lista_destinatarios = [
                correo.strip() for correo in destinatarios
                if correo.strip() and "@" in correo.strip()
            ]
            print(f"input cribado:{str(lista_destinatarios)}")
            #lista_targets = [item.strip() for item in destinatarios.split(",")]
            data = generar_jsonMails(user, lista_destinatarios)
            print(f"data:{str(data)}")
            url = "https://apisolicitudesonlinepre.cinconet.local/api/Solicitud"
            headers = {
                    'Content-Type': 'application/json;charset=UTF-8',
                    'Authorization': f'Bearer {validToken}'
                }
            response = requests.post(url, data=json.dumps(data), headers=headers, verify=False)
        except requests.exceptions.RequestException as e:
            resultado+= f"⚠️ Error al enviar la solicitud de office365: {str(e)}"
    if destinatarios_externos:
        destinatarios=destinatarios_externos
        print(f"input usuarios:{destinatarios_externos}")
        user = "vchumillas@megamedia.es"
        validToken = generarToken(user)
        
        url = "https://apisolicitudesonlinepre.cinconet.local/api/Recurso/Carrito"

        payload_dict = {
        "idRecurso": 0,
        "idRecursoCatalogo": 161,
        "nombre": "Interplay (Usuarios red de programas (RDP) - Telecinco)",
        "descripcion": "Inteplay (Usuarios red de programas Telecinco)",
        "urlImagen": "NO_IMAGE",
        "coste": 0.00,
        "activo": True,
        "nivelesRecurso": [
            {
                "id": 28227,
                "nivel1Id": id_programa,
                "nivel1": None,
                "nivel2Id": id_rol,
                "nivel2": None,
                "nivel3": None
            }
        ],
        "niveles": [
            {
                "id": 249,
                "descripcion": "Supervivientes",
                "nivel2": [
                    {
                        "id": 160,
                        "descripcion": "Media manager",
                        "nivel3": ""
                    },
                    {
                        "id": 161,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 393,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 394,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 395,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 409,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 502,
                "descripcion": "Fiesta",
                "nivel2": [
                    {
                        "id": 251,
                        "descripcion": "Media Manager",
                        "nivel3": ""
                    },
                    {
                        "id": 385,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 389,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 391,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 397,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 408,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 534,
                "descripcion": "La vida sin filtros",
                "nivel2": [
                    {
                        "id": 279,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 383,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 384,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 388,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 407,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 535,
                "descripcion": "Así es la vida",
                "nivel2": [
                    {
                        "id": 386,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 387,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 390,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 396,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 406,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 568,
                "descripcion": "Vamos A Ver",
                "nivel2": [
                    {
                        "id": 374,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 375,
                        "descripcion": "Guionista",
                        "nivel3": ""
                    },
                    {
                        "id": 376,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 377,
                        "descripcion": "Producción",
                        "nivel3": ""
                    },
                    {
                        "id": 398,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 403,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 405,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 411,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 569,
                "descripcion": "La Mirada Crítica",
                "nivel2": [
                    {
                        "id": 370,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 371,
                        "descripcion": "Guionista",
                        "nivel3": ""
                    },
                    {
                        "id": 372,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 373,
                        "descripcion": "Producción",
                        "nivel3": ""
                    },
                    {
                        "id": 399,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 400,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    },
                    {
                        "id": 401,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 570,
                "descripcion": "TardeAR",
                "nivel2": [
                    {
                        "id": 366,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 367,
                        "descripcion": "Guionista",
                        "nivel3": ""
                    },
                    {
                        "id": 368,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 369,
                        "descripcion": "Producción",
                        "nivel3": ""
                    },
                    {
                        "id": 402,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 404,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 410,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 615,
                "descripcion": "Vecinos",
                "nivel2": [
                    {
                        "id": 510,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 511,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 512,
                        "descripcion": "Guionista",
                        "nivel3": ""
                    },
                    {
                        "id": 513,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 514,
                        "descripcion": "Producción",
                        "nivel3": ""
                    },
                    {
                        "id": 515,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 516,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    }
                ]
            },
            {
                "id": 683,
                "descripcion": "El Programa de Ana Rosa",
                "nivel2": [
                    {
                        "id": 557,
                        "descripcion": "Realizador/Ayudante de Realización",
                        "nivel3": ""
                    },
                    {
                        "id": 558,
                        "descripcion": "Redactor",
                        "nivel3": ""
                    },
                    {
                        "id": 559,
                        "descripcion": "Guionista",
                        "nivel3": ""
                    },
                    {
                        "id": 560,
                        "descripcion": "Dirección",
                        "nivel3": ""
                    },
                    {
                        "id": 561,
                        "descripcion": "Producción",
                        "nivel3": ""
                    },
                    {
                        "id": 562,
                        "descripcion": "Postproducción",
                        "nivel3": ""
                    },
                    {
                        "id": 563,
                        "descripcion": "Editor de vídeo",
                        "nivel3": ""
                    }
                ]
            }
        ],
        "campos": [
            {
                "id": 113611,
                "maestroId": 10,
                "valor": observaciones or None,
                "recursoId": 0,
                "obligatorio": False
            },
            {
                "id": 113612,
                "maestroId": 12,
                "valor": None,
                "recursoId": 0,
                "obligatorio": False
            }
        ],
        "baja": False,
        "bajaCatalogo": True,
        "descargaPlano": False,
        "nivel1": "Tipo de acceso",
        "obligatorioNivel1": True,
        "nivel2": "Permisos",
        "obligatorioNivel2": True,
        "nivel3": None,
        "obligatorioNivel3": False
    }

        headers = {
            'Content-Type': 'application/json;charset=UTF-8',
            'Authorization': f'Bearer {validToken}'
        }

        try:
            response = requests.post(url, headers=headers, json=payload_dict, verify=False)
            response.raise_for_status()

            result = response.json()
            solicitud_id = result.get("id", "N/A")
            
            #lista_targets = [item.strip() for item in destinatarios.split(",")]
            data = generar_jsonMails_externos(user, destinatarios)
            print(f"data:{str(data)}")
            url = "https://apisolicitudesonlinepre.cinconet.local/api/Solicitud"
            headers = {
                    'Content-Type': 'application/json;charset=UTF-8',
                    'Authorization': f'Bearer {validToken}'
                }
            response = requests.post(url, data=json.dumps(data), headers=headers, verify=False)
            nombres=""
            for destinatario in destinatarios:
                nombres+=destinatario["nombre"]+" "+destinatario["apellidos"] + ", "
        except requests.exceptions.RequestException as e:
            resultado+= f"⚠️ Error al enviar la solicitud de Office365: {str(e)}"

        lista_destinatarios=", ".join(lista_destinatarios)
    return (
        f"📡 Solicitud de Inews enviada correctamente:\n"
        f"- tipo de Acceso: {tipoAcceso}\n"
        f"- permisos: {permisos}\n"
        f"- observaciones: {observaciones or 'sin observaciones'}\n"
        f"- destinatarios: {nombres + str(lista_destinatarios) or 'tu'}\n"
    )

def create_solicitud_Portatil(descripcion,cc,fecha="",destinatarios_internos="", destinatarios_externos=None):
    lista_destinatarios=[]
    nombres=""
    if destinatarios_internos:
        destinatarios=destinatarios_internos
        destinatarios = [
            "vchumillas@megamedia.es" if d.strip().lower() == "user" else d.strip()
            for d in destinatarios
        ]
        print(f"input usuarios:{destinatarios_internos}")
        user = "vchumillas@megamedia.es"
        validToken = generarToken(user)
        
        url = "https://apisolicitudesonlinepre.cinconet.local/api/Recurso/Carrito"


        payload_dict = {
            "idRecurso": 0,
            "idRecursoCatalogo": 264,
            "nivelesRecurso": [],
            "campos": [
            {
                "id": 113603,
                "maestroId": 2,
                "valor": cc,
                "recursoId": 0,
                "obligatorio": True
            },
            {
                "id": 113604,
                "maestroId": 4,
                "valor": descripcion,
                "recursoId": 0,
                "obligatorio": True
            },
            {
                "id": 113605,
                "maestroId": 7,
                "valor": fecha or None,
                "recursoId": 0,
                "obligatorio": False
            }
        ],
            "baja": False,
        }

        headers = {
            'Content-Type': 'application/json;charset=UTF-8',
            'Authorization': f'Bearer {validToken}'
        }

        try:
            response = requests.post(url, headers=headers, json=payload_dict, verify=False)
            response.raise_for_status()

            result = response.json()
            solicitud_id = result.get("id", "N/A")
            lista_destinatarios = [
                correo.strip() for correo in destinatarios
                if correo.strip() and "@" in correo.strip()
            ]
            print(f"input cribado:{str(lista_destinatarios)}")
            #lista_targets = [item.strip() for item in destinatarios.split(",")]
            data = generar_jsonMails(user, lista_destinatarios)
            print(f"data:{str(data)}")
            url = "https://apisolicitudesonlinepre.cinconet.local/api/Solicitud"
            headers = {
                    'Content-Type': 'application/json;charset=UTF-8',
                    'Authorization': f'Bearer {validToken}'
                }
            response = requests.post(url, data=json.dumps(data), headers=headers, verify=False)
        except requests.exceptions.RequestException as e:
            resultado+= f"⚠️ Error al enviar la solicitud de office365: {str(e)}"
    if destinatarios_externos:
        destinatarios=destinatarios_externos
        print(f"input usuarios:{destinatarios_externos}")
        user = "vchumillas@megamedia.es"
        validToken = generarToken(user)
        
        url = "https://apisolicitudesonlinepre.cinconet.local/api/Recurso/Carrito"

        payload_dict = {
            "idRecurso": 0,
            "idRecursoCatalogo": 264,
            "nivelesRecurso": [],
            "campos": [
            {
                "id": 113603,
                "maestroId": 2,
                "valor": cc,
                "recursoId": 0,
                "obligatorio": True
            },
            {
                "id": 113604,
                "maestroId": 4,
                "valor": descripcion,
                "recursoId": 0,
                "obligatorio": True
            },
            {
                "id": 113605,
                "maestroId": 7,
                "valor": fecha or None,
                "recursoId": 0,
                "obligatorio": False
            }
        ],
            "baja": False,
        }

        headers = {
            'Content-Type': 'application/json;charset=UTF-8',
            'Authorization': f'Bearer {validToken}'
        }

        try:
            response = requests.post(url, headers=headers, json=payload_dict, verify=False)
            response.raise_for_status()

            result = response.json()
            solicitud_id = result.get("id", "N/A")
            
            #lista_targets = [item.strip() for item in destinatarios.split(",")]
            data = generar_jsonMails_externos(user, destinatarios)
            print(f"data:{str(data)}")
            url = "https://apisolicitudesonlinepre.cinconet.local/api/Solicitud"
            headers = {
                    'Content-Type': 'application/json;charset=UTF-8',
                    'Authorization': f'Bearer {validToken}'
                }
            response = requests.post(url, data=json.dumps(data), headers=headers, verify=False)
            nombres=""
            for destinatario in destinatarios:
                nombres+=destinatario["nombre"]+" "+destinatario["apellidos"] + ", "
        except requests.exceptions.RequestException as e:
            resultado+= f"⚠️ Error al enviar la solicitud de Office365: {str(e)}"

        lista_destinatarios=", ".join(lista_destinatarios)
    return (
        f"📡 Solicitud de portatil enviada correctamente:\n"
        f"- descripcion: {descripcion}\n"
        f"- centro de coste: {cc}\n"
        f"- fecha: {fecha or 'Sin fecha'}\n"
        f"- destinatarios: {nombres + str(lista_destinatarios) or 'tu'}\n"
    )


In [31]:
programa = "La vida sin filtros"
rol = "Postproducción"
create_solicitud_Portatil("testing descripcion","centro coste chulo","",["cmangisch@megamedia.es"],None)

input usuarios:['cmangisch@megamedia.es']


c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonlinepre.cinconet.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonlinepre.cinconet.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonlinepre.cinconet.local'. Adding certificate verification i

input cribado:['cmangisch@megamedia.es']
vchumillas@megamedia.es
Usuario: vchumillas
Dominio: megamedia.es


c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonline.cinconet.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonlinepre.cinconet.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonlinepre.cinconet.local'. Adding certificate verification is s

cmangisch@megamedia.es
Usuario: cmangisch
Dominio: megamedia.es


c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonline.cinconet.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonlinepre.cinconet.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonlinepre.cinconet.local'. Adding certificate verification is s

data:{'autorizadorId': 2248, 'autorizadorApoyoId': None, 'telefono': '636462636', 'departamento': 'DESARROLLO, PRODUCTO E INNOVACION', 'destinatarios': [{'nombre': 'CECILIA', 'apellidos': 'MANGISCH .', 'login': 'cmangisch', 'email': 'cmangisch@megamedia.es', 'telefono': '671523508', 'departamento': 'IT', 'empresa': 'MEGAMEDIA TELEVISION, S.L.', 'centro': 'Fuencarral', 'edificio': 'E', 'planta': '1'}], 'observaciones': 'test', 'justificacion': 'Test', 'productoraId': None, 'programa': '', 'centro': 'Fuencarral', 'edificio': 'H', 'planta': '0'}


"📡 Solicitud de portatil enviada correctamente:\n- descripcion: testing descripcion\n- centro de coste: centro coste chulo\n- fecha: Sin fecha\n- destinatarios: ['cmangisch@megamedia.es']\n"

In [15]:
mapeo = {
    "Supervivientes": {
        "id": 249,
        "roles": {
            "Media manager": 160,
            "Redactor": 161,
            "Dirección": 393,
            "Editor de vídeo": 394,
            "Realizador/Ayudante de Realización": 395,
            "Postproducción": 409
        }
    },
    "Fiesta": {
        "id": 502,
        "roles": {
            "Media Manager": 251,
            "Dirección": 385,
            "Editor de vídeo": 389,
            "Redactor": 391,
            "Realizador/Ayudante de Realización": 397,
            "Postproducción": 408
        }
    },
    "La vida sin filtros": {
        "id": 534,
        "roles": {
            "Editor de vídeo": 279,
            "Redactor": 383,
            "Realizador/Ayudante de Realización": 384,
            "Dirección": 388,
            "Postproducción": 407
        }
    },
    "Así es la vida": {
        "id": 535,
        "roles": {
            "Redactor": 386,
            "Realizador/Ayudante de Realización": 387,
            "Dirección": 390,
            "Editor de vídeo": 396,
            "Postproducción": 406
        }
    },
    "Vamos A Ver": {
        "id": 568,
        "roles": {
            "Redactor": 374,
            "Guionista": 375,
            "Dirección": 376,
            "Producción": 377,
            "Postproducción": 398,
            "Dirección (2)": 403,
            "Realizador/Ayudante de Realización": 405,
            "Editor de vídeo": 411
        }
    },
    "La Mirada Crítica": {
        "id": 569,
        "roles": {
            "Redactor": 370,
            "Guionista": 371,
            "Dirección": 372,
            "Producción": 373,
            "Postproducción": 399,
            "Editor de vídeo": 400,
            "Realizador/Ayudante de Realización": 401
        }
    },
    "TardeAR": {
        "id": 570,
        "roles": {
            "Redactor": 366,
            "Guionista": 367,
            "Dirección": 368,
            "Producción": 369,
            "Realizador/Ayudante de Realización": 402,
            "Postproducción": 404,
            "Editor de vídeo": 410
        }
    },
    "Vecinos": {
        "id": 615,
        "roles": {
            "Realizador/Ayudante de Realización": 510,
            "Redactor": 511,
            "Guionista": 512,
            "Dirección": 513,
            "Producción": 514,
            "Postproducción": 515,
            "Editor de vídeo": 516
        }
    },
    "El Programa de Ana Rosa": {
        "id": 683,
        "roles": {
            "Realizador/Ayudante de Realización": 557,
            "Redactor": 558,
            "Guionista": 559,
            "Dirección": 560,
            "Producción": 561,
            "Postproducción": 562,
            "Editor de vídeo": 563
        }
    }
}

def obtener_ids(programa, rol):
    if programa not in mapeo:
        return f"Programa no encontrado: {programa}"
    nivel1_id = mapeo[programa]["id"]
    nivel2_id = mapeo[programa]["roles"].get(rol)
    if nivel2_id is None:
        return f"Rol no encontrado: {rol} en el programa {programa}"
    id_programa=nivel1_id
    id_rol=nivel2_id
    return id_programa,id_rol

# Ejemplo de uso:
id_programa,id_rol=obtener_ids("Así es la vida", "Dirección")
print(id_programa)
print(id_rol)



535
390
